<a href="https://colab.research.google.com/github/thessafuzaro/Trabalho_Imagens/blob/main/Classificacao_Imagens_MLPClassifier_GoogleDrive.ipynb" target="_parent"><img src="https://colab.research.google.com/assets/colab-badge.svg" alt="Open In Colab"/></a>

In [25]:
#Segundo Projeto de redes neurais
#Processamento de Imagens
#Aluna: Thessa Fuzaro Mendes
#Data: 06/07/2021

In [26]:
from matplotlib import pyplot as plt  # para fazer graficos
import numpy as np
import pandas as pd
import cv2  #biblioteca que trabalha com imagens
import random
import zipfile  #para trabalhar com arquivos zip
from sklearn.neural_network import MLPClassifier
from sklearn import preprocessing
from sklearn.model_selection import train_test_split
from sklearn.metrics import classification_report

In [27]:
#Importando os arquivos que estão no googledrive
!gdown --id 1HNVtQ_e03o7QqkdQLtDYKsJNfW3Bv403/view?usp=sharing

Permission denied: https://drive.google.com/uc?id=1HNVtQ_e03o7QqkdQLtDYKsJNfW3Bv403/view?usp=sharing
Maybe you need to change permission over 'Anyone with the link'?


In [28]:
#Extraindo os arquivos zip e salvando na variavel x
Arq =['All_Flowers.zip']  #importando os arquivos
Image_Size=80
Image_Name=[]  #lista em branco
X_Gray=[]  
X_Original = []
for FILE in Arq:
  file_name = zipfile.ZipFile(FILE, 'r')
  file_name.extractall()    #extraindo os arquivos e colocando em uma lista unica com os nomes
  with zipfile.ZipFile(FILE, "r") as f:
      for Name in f.namelist():
          Image_Name.append(Name)
         # print(name)
          X_Gray.append(cv2.resize(cv2.imread(Name, 0),(Image_Size,Image_Size))) #passando a imagem para preto e branco e redimensionando

FileNotFoundError: ignored

In [ ]:
X_Gray[0].shape

In [ ]:
#Recuperando o nome com 4 primeiras letras
Y_Names = []
for yy in Image_Name:
  Name = yy[0]+yy[1]+yy[2]+yy[3]
  Y_Names.append(Name)
  #print(Y_Names)

In [ ]:
#Plotando algumas imagens em preto e branco com seus respectivos nomes (4 primeiras letras)
num = len(Y_Names)
plt.figure(figsize=(16,16))
for i in range (16) :
  ax = plt.subplot(4,4,i+1)
  kk = random.randrange(num)
  plt.imshow(X_Gray[kk], cmap='gray')
  plt.title(Y_Names[kk])
  plt.axis("off")

In [ ]:
Unique_List = []      #gera uma lista unica 
for yy in Y_Names:
  if yy not in Unique_List:
    Unique_List.append(yy)

In [ ]:
n = len(Unique_List)  #separa em classes ao inves de nome aparece o índice referente
Y = []
for yy in Y_Names:
  for i in range(n):
    if(yy == Unique_List[i]):
      Y.append(i)

In [ ]:
Unique_List #mostrandoa lista unica

In [ ]:
X_Data=[]
for xx in X_Gray:  #lendo  as fotos em preto e branco 
  X_Data.append(xx.ravel())  # cada linha é uma variável dependente 

X = pd.DataFrame(X_Data)

In [ ]:
#Separando os dados em 80% de treino e 20% de teste e misturando os dados
X_Train, X_Test, Y_Train, Y_Test = train_test_split(X,Y, test_size=0.40, shuffle=True, random_state=3)

In [ ]:
#Escalonando os dados entre -1 e 1, usando a função preprocessing:
Scaler_X = preprocessing.MinMaxScaler((-1,1)).fit(X)
X_Train = Scaler_X.transform(X_Train)
X_Test = Scaler_X.transform(X_Test)

In [ ]:
#Definindo a rede neural:
model = MLPClassifier(solver='lbfgs',activation='tanh',alpha=1e-5,hidden_layer_sizes=(20),
                      max_iter=1000,random_state=1)
model.fit(X_Train,Y_Train) #Treinando a rede neural

In [ ]:
#Calculando treino e teste por meio da rede neural
Y_Calc_Train=model.predict(X_Train)
Y_Calc_Test=model.predict(X_Test)

In [ ]:
#Fazendo a matriz confusão para treino
df = pd.DataFrame({'True':Y_Train, 'Calc':Y_Calc_Train})
print(pd.crosstab(df['True'],df['Calc']))

In [ ]:
#Fazendo a matriz confusão para teste
df = pd.DataFrame({'True':Y_Test, 'Calc':Y_Calc_Test})
print(pd.crosstab(df['True'],df['Calc']))

In [ ]:
#Chamando a métrica Classification Report
Y_True = Y_Test #Amazenando os dados de teste em Y_True
Y_Pred = Y_Calc_Test #Armazenando os dados calculados para teste em Y_Pred
Title = ['Tulip','Sunflower','Dandelion','Rose','Daisy']
print(classification_report(Y_True, Y_Pred,target_names=Title))